In [1]:
from sqlalchemy import create_engine, Column, ForeignKey
from sqlalchemy.types import Integer, String, DateTime, Numeric, Boolean
from sqlalchemy.orm import sessionmaker, relationship, backref
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

In [2]:
engine = create_engine("sqlite:///:memory:")
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [3]:
class Cookie(Base):
    __tablename__ = "cookies"
    
    cookie_id = Column(Integer(), primary_key=True)
    cookie_name = Column(String(100), nullable=False, unique=True, index=True)
    quantity = Column(Integer(), default=0)
    unit_cost = Column(Numeric(12, 2))

    def __repr__(self):
        return f"<Cookie: {self.cookie_name}>"


class User(Base):
    __tablename__ = "users"

    user_id = Column(Integer(), primary_key=True)
    username = Column(String(100), unique=True, nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)

    def __repr__(self):
        return f"<User: {self.username}>"


class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey("users.user_id"))
    
    user = relationship("User", backref=backref("orders"), order_by=order_id)


class LineItem(Base):
    __tablename__ = "lineitems"

    line_item_id = Column(Integer(), primary_key=True)
    cookie_id = Column(Integer(), ForeignKey("cookies.cookie_id"))
    order_id = Column(Integer(), ForeignKey("orders.order_id"))
    quantity = Column(Integer(), nullable=False)

    order = relationship("Order", backref=backref("line_items"))
    cookie = relationship("Cookie", backref=backref("line_items"))



In [4]:
Base.metadata.create_all(bind=engine)

In [5]:
chocolate_chip = Cookie(cookie_name="chocolate chip",
                        quantity=12,
                        unit_cost=0.5)

session.add(chocolate_chip)
session.commit()